## Problem: I am too slow in parsing large pgns





## Overview of observed times.

#### In parse_databases.ipynb

%time data_elite_10000_10 = parse_database(elite_pgn,10000,10)

shows a time of 1min 57 sec, for 10000 games.

This means 85 games per second. 

---

%time data_elite_1000_6 = parse_database(elite_pgn,1000,6)


shows a time of 7.86s, for 1000 games.\

This means 120 games per second. 

#### In lichess-full-analysis.ipynb

%time data_shortlichess_100000_16 = parse_database(shortlichess_pgn,100000,16)

shows a time of 32min 41s


This means 50 games per second.

### Comments

Possible causes: overuse of the function Game.board() of python-chess

## The parsepgn code

def parse_database(pgn,num_games,ply_depth):
    
    dictionary = {}
    
    for i in range(num_games):

        game = chess.pgn.read_game(pgn)
        
        if game == None:
            break
            
        parse_opening_game(dictionary,game,ply_depth)

    return dictionary


In [1]:
#required
import chess
import chess.pgn
import re
import math
import os #for os.path.basename() only 

#source code
import sys
sys.path.append('../src')
import chessposition
import parsepgn
import databaseprune
import analysisbasic
import display


import lightdb
from lightdb import LightDatabase

# import specific methods
from chessposition import *
from parsepgn import *
from databaseprune import *
from analysisbasic import *
from display import *

# overview of the functions that are really used
from parsepgn import parse_new_game

In [2]:
import fastparseclass
from fastparseclass import *

In [3]:

#open a pgn file
myPath="C:/Users/User/Documents/60-69_Learning/63 Chess/"

pgn = open(myPath+"shortlichess-16ply-detag-100000games.pgn")

In [4]:
%%time

for i in range(1000):
    game = chess.pgn.read_game(pgn)

Wall time: 988 ms


## Test Fastparse and parse

In [5]:
%%time
db=parse_database(pgn,100,10)
len(db.keys())

Wall time: 10.2 s


5938

In [6]:
%%time
fastdb = fastparse_database(pgn,1000,10)
len(fastdb.all_positions)

Wall time: 1.2 s


5081

In [10]:
%%time 
fastenrich_database(pgn,10000,10,fastdb)
len(fastdb.all_positions)

Wall time: 10.4 s


5081

## Play with pseudoparse functions

To find the bottleneck of the parse method. It seems, it is in the update method of the class ChessPosition.

In [4]:
%%time

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()

Wall time: 1.33 s


In [5]:
%%time

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()
    ply=0
    for move in game.mainline_moves():
        ply+=1
        board.push(move)
        if ply ==10:
            break

Wall time: 1.29 s


In [6]:
def pseudo_parse_opening_game(new_game,depth):

    new_position = new_game

    for move in new_game.mainline_moves(): 
        
        if new_position.ply() == depth:
            break
        new_position = new_position.variation(move)

In [7]:
    
%%time

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()
    ply=0
    
    pseudo_parse_opening_game(game,10)

Wall time: 1.24 s


In [8]:
def pseudo_parse_with_id(new_game,depth):
    
    new_position = new_game

    new_id = gamenode_to_movestack(new_position)

    for move in new_game.mainline_moves(): 
        
        if new_position.ply() == depth:
            break
        new_position = new_position.variation(move)


In [9]:
%%time

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()
    ply=0
    
    pseudo_parse_with_id(game,10)

Wall time: 1.2 s


In [10]:
def pseudo_parse_with_dictionary_write(dictionary,new_game,depth):
    
    new_position = new_game

    new_id = gamenode_to_movestack(new_position)
    
    
    for move in new_game.mainline_moves(): 
        
        if new_position.ply() == depth:
            break
        new_position = new_position.variation(move)
        
    dictionary[new_id] = game


In [11]:
%%time

dizionario = {}

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()
    ply=0
    
    pseudo_parse_with_dictionary_write (dizionario,game,10)

Wall time: 1.14 s


In [12]:
def pseudo_parse_with_dictionary_lookup(dictionary,dictionary2,new_game,depth):
    
    new_position = new_game

    new_id = gamenode_to_movestack(new_position)
    
    
    for move in new_game.mainline_moves(): 
        
        if new_position.ply() == depth:
            break
        new_position = new_position.variation(move)
        
    if new_id in dictionary.keys():
        dictionary[new_id] = game
    else: 
        dictionary2[new_id] = game


In [13]:
%%time

dizionario1 = {}
dizionario2 = {}

for i in range(1000):
    game = chess.pgn.read_game(pgn)
    board = game.board()
    ply=0
    
    pseudo_parse_with_dictionary_lookup(dizionario1,dizionario2,game,10)

Wall time: 1.31 s


In [14]:
%%time
parse_database(pgn,1000,10)

Wall time: 10.8 s


{'': <chessposition.ChessPosition at 0x21c66b418b0>,
 'd2d4': <chessposition.ChessPosition at 0x21c66b85b50>,
 'd2d4d7d5': <chessposition.ChessPosition at 0x21c66b856d0>,
 'd2d4d7d5c1f4': <chessposition.ChessPosition at 0x21c66afcdf0>,
 'd2d4d7d5c1f4e7e6': <chessposition.ChessPosition at 0x21c66b65160>,
 'd2d4d7d5c1f4e7e6g1f3': <chessposition.ChessPosition at 0x21c66bd4400>,
 'd2d4d7d5c1f4e7e6g1f3f7f6': <chessposition.ChessPosition at 0x21c66ad6940>,
 'd2d4d7d5c1f4e7e6g1f3f7f6e2e3': <chessposition.ChessPosition at 0x21c66b15f10>,
 'd2d4d7d5c1f4e7e6g1f3f7f6e2e3g7g5': <chessposition.ChessPosition at 0x21c66a53910>,
 'd2d4d7d5c1f4e7e6g1f3f7f6e2e3g7g5f4g3': <chessposition.ChessPosition at 0x21c66b41a60>,
 'd2d4d7d5c1f4e7e6g1f3f7f6e2e3g7g5f4g3h7h5': <chessposition.ChessPosition at 0x21c66bd4580>,
 'g1f3': <chessposition.ChessPosition at 0x21c66b3b460>,
 'g1f3c7c5': <chessposition.ChessPosition at 0x21c66a1abe0>,
 'g1f3c7c5g2g3': <chessposition.ChessPosition at 0x21c66b8a2b0>,
 'g1f3c7c5g2g3

### First conclusions. 

Even with database lookup and id generation (which itself involves board().move_stack) we are parsing 1000 games in 1.61s.

This is 621 games per second.

If instead I call ChessPosition and update, then the time becomes 9.44s for 1000 games

This is only 100 games per second. 



Thus, the bottleneck is somewhere in ChessPosition.update() I guess..! 

We should be able to increase the performance sixfold. Not bad, This means that a 1h computation becomes a 10 min computation


By the way, the variability in the above estimates is huge, sometimes 1.61s become 1.03s or 2s, instead the 9.44 sometimes becomes 9.2, sometimes 10.1s etc. 


---


Even a speed of 500 games per second would not be too bad. 1000 games in 2 seconds, 1.000.000 games in less than 1 hour

Probably, calling "my fen id" and "board().gamestack" and next_position to calculate the variation, is not a good idea.

#### First possible fix:

just write a light_parse_database that parses doing only necessary reads. 

## Trying other libraries

In [1]:
#import pgn_parser
#from pgn_parser import pgn, parser

# this i dont know how to read a file

In [2]:
import pgn as renatopgn

#renatopp

#docs stackoverflow 6512047

In [3]:
pgn_text = open("../pgn/C25.pgn").read()

#pgn.loads(pgn_text) is a list of PGNGame

games = renatopgn.loads(pgn_text)

AttributeError: module 'pgn' has no attribute 'loads'